This explains how I implemented the image transformations in `ImageLab/src/components/ImageCanvas.tsx`. I present each transform as an algorithm, prove its correctness, and explain my design decisions.

## 1. Color Space and Design

### 1.1 sRGB

All modern displays, web browsers, and the HTML Canvas API work in sRGB color space. sRGB values are gamma-encoded. They don't represent linear light intensity, but rather perceptually-uniform brightness levels optimized for human vision.

This means that...
- Values range from 0 to 255 for each channel (R, G, B)
- The relationship to linear light is non-linear: $R_{linear} \approx (R_{sRGB}/255)^{2.2}$
- Value 128 appears as perceptual "middle gray" on screen (not 50% light intensity)
- Gamma encoding makes efficient use of 8 bits by allocating more values to darker tones where human eyes are more sensitive

### 1.2 Design Decisions

I chose to perform all transformations in sRGB space (without linearization because):

1. Linearizing and re-encoding requires power operations per pixel which are expensive and computation matters as we are working with large images
2. For real-time image editing, gamma-space operations provide visually acceptable results
3. Canvas ImageData provides sRGB values

This means most transforms work on gamma-encoded values. I've ensured each algorithm is mathematically sound in this space.

As I am most comfortable with Python, I originally thought it would be most computationally efficient to leverage libraries for the matrix operations. However, when looking through the documentation for libraries like ml-matrix or numeric.js, I noticed they require individual function calls for each operation. Unlike NumPy, which delegates to optimized C/Fortran code and processes batched operations in compiled code, JavaScript matrix libraries execute in the JavaScript runtime. For our use case—applying a 3×3 transformation to 2+ million pixels—this means 2+ million separate function calls, each with JavaScript overhead, object allocations, and garbage collection pressure.

Therefore, I write all matrix operations inline by extracting matrix elements and computing per pixel.

### 1.3 Mathematical Notation

- $\mathbf{r} = [R, G, B]^T$ represents a pixel in gamma-encoded sRGB space, where each component is in $[0, 255]$
- $I$ denotes the $3 \times 3$ identity matrix
- $M$ denotes a $3 \times 3$ transformation matrix
- $\mathbf{o}$ denotes a $3 \times 1$ offset vector
- An affine transform has the form: $\mathbf{r}' = M\mathbf{r} + \mathbf{o}$
- All operations include implicit clamping to $[0, 255]$





## 2. Brightness Adjustment

**What I wanted:** A transform that shifts all color channels by the same amount, making the image uniformly lighter or darker.

### Algorithm 2.1: Brightness Transform

```
Input: pixel r = [R, G, B]ᵀ, brightness offset b ∈ ℝ
Output: transformed pixel r' = [R', G', B']ᵀ

1. R' ← clamp(R + b, 0, 255)
2. G' ← clamp(G + b, 0, 255)  
3. B' ← clamp(B + b, 0, 255)
4. return [R', G', B']ᵀ
```

**Affine form:** $\mathbf{r}' = I\mathbf{r} + b\mathbf{1}$ where $\mathbf{1} = [1, 1, 1]^T$

**Implementation:** See `buildBrightnessMatrix()` at lines 40-49 in `ImageCanvas.tsx`:
```typescript
const buildBrightnessMatrix = (value: number): { matrix: number[]; offset: number[] } => {
  const matrix = [1, 0, 0, 0, 1, 0, 0, 0, 1]; // Identity matrix
  const offset = [value, value, value]; // Uniform offset
  return { matrix, offset };
};
```

### Proof of Correctness

**Claim:** This transform shifts all channels uniformly while preserving color relationships.

**Proof:**
For any two pixels $\mathbf{r}_1, \mathbf{r}_2$:
$$\mathbf{r}'_1 - \mathbf{r}'_2 = (I\mathbf{r}_1 + b\mathbf{1}) - (I\mathbf{r}_2 + b\mathbf{1}) = I(\mathbf{r}_1 - \mathbf{r}_2) = \mathbf{r}_1 - \mathbf{r}_2$$

The difference between any two pixels remains constant, preserving relative color relationships. The transform is a translation in RGB space. 





## 3. Contrast Adjustment

**What I wanted:** A transform that scales distances from middle gray (128), making dark regions darker and bright regions brighter while keeping middle gray fixed.

### Algorithm 3.1: Contrast Transform

```
Input: pixel r = [R, G, B]ᵀ, contrast factor c ∈ ℝ⁺
Output: transformed pixel r'

1. For each channel k ∈ {R, G, B}:
2.   k' ← 128 + c(k - 128)
3.   k' ← clamp(k', 0, 255)
4. return r'
```

**Affine form:** $\mathbf{r}' = cI\mathbf{r} + 128(1-c)\mathbf{1}$

**Implementation:** See `buildContrastMatrix()` at lines 53-62 in `ImageCanvas.tsx`:
```typescript
const buildContrastMatrix = (value: number): { matrix: number[]; offset: number[] } => {
  const matrix = [value, 0, 0, 0, value, 0, 0, 0, value]; // Scale by c
  const offset = [128 * (1 - value), 128 * (1 - value), 128 * (1 - value)];
  return { matrix, offset };
};
```

### Proof of Correctness

**Claim:** Middle gray (128) is a fixed point, and distances from middle gray scale by factor $c$.

**Proof:**
Let $\mathbf{g} = [128, 128, 128]^T$ represent middle gray in sRGB.

**(1) Fixed point property:**
$$\mathbf{g}' = cI\mathbf{g} + 128(1-c)\mathbf{1} = c \cdot 128 + 128(1-c) = 128 = \mathbf{g}$$

**(2) Distance scaling:**
For any pixel $\mathbf{r}$, the distance from middle gray after transformation is:
$$\mathbf{r}' - \mathbf{g} = (cI\mathbf{r} + 128(1-c)\mathbf{1}) - 128\mathbf{1} = c(\mathbf{r} - 128\mathbf{1}) = c(\mathbf{r} - \mathbf{g})$$

Thus distances from middle gray scale by exactly $c$. 

*Note:* In sRGB, the value 128 represents perceptual middle gray (approximately 50% brightness as perceived by human vision), making it the natural anchor point for contrast adjustments.



## 4. Saturation Adjustment

**What I wanted:** A transform that blends each color with its grayscale equivalent, controlling color intensity while preserving perceived brightness.

### Algorithm 4.1: Saturation Transform

```
Input: pixel r = [R, G, B]ᵀ, saturation factor s ∈ ℝ
Output: transformed pixel r'

1. L ← 0.299·R + 0.587·G + 0.114·B    // Gamma-space luminance
2. gray ← [L, L, L]ᵀ
3. r' ← gray·(1 - s) + r·s             // Linear interpolation
4. return clamp(r', 0, 255)
```

**Affine form:** This can be expressed as $\mathbf{r}' = M_s\mathbf{r}$ where:

$$M_s = \begin{bmatrix}
w_R + (1-w_R)s & w_G(1-s) & w_B(1-s) \\
w_R(1-s) & w_G + (1-w_G)s & w_B(1-s) \\
w_R(1-s) & w_G(1-s) & w_B + (1-w_B)s
\end{bmatrix}$$

where $(w_R, w_G, w_B) = (0.299, 0.587, 0.114)$ are the **Rec.601 luminance weights**.

**Implementation:** See `buildSaturationMatrixGamma()` at lines 68-95 in `ImageCanvas.tsx`:
```typescript
const buildSaturationMatrixGamma = (saturation: number): number[] => {
  if (saturation === 1) return [1, 0, 0, 0, 1, 0, 0, 0, 1];
  
  const wR = 0.299, wG = 0.587, wB = 0.114;
  const s = saturation;
  
  return [
    wR + (1 - wR) * s, wG * (1 - s), wB * (1 - s),
    wR * (1 - s), wG + (1 - wG) * s, wB * (1 - s),
    wR * (1 - s), wG * (1 - s), wB + (1 - wB) * s
  ];
};
```

### Proof of Correctness

**Claim:** The saturation transform (1) preserves gamma-space luminance and (2) leaves gray pixels invariant.

**Proof:**

**(1) Gamma-space luminance preservation:**

The gamma-space luminance of the transformed pixel is:
$$L' = w_R R' + w_G G' + w_B B'$$

Substituting $\mathbf{r}' = L\mathbf{1}(1-s) + \mathbf{r} \cdot s$:
$$L' = w_R[L(1-s) + Rs] + w_G[L(1-s) + Gs] + w_B[L(1-s) + Bs]$$
$$= L(1-s)(w_R + w_G + w_B) + s(w_R R + w_G G + w_B B)$$
$$= L(1-s) \cdot 1 + s \cdot L = L$$

The gamma-space luminance is preserved. 

**(2) Gray invariance:**

Let $\mathbf{r} = [g, g, g]^T$ be a gray pixel. Then:
$$L = w_R g + w_G g + w_B g = g(w_R + w_G + w_B) = g$$

Thus:
$$\mathbf{r}' = [g, g, g](1-s) + [g, g, g]s = [g, g, g]$$

Gray pixels remain unchanged for all $s$. 


I use Rec.601 weights $(0.299, 0.587, 0.114)$ rather than the more modern Rec.709 weights $(0.2126, 0.7152, 0.0722)$ because Rec.709 weights are designed for linear light. Rec.601 weights are empirically better for gamma space. 

However, there is no perfect set of constant weights for gamma-space luminance because gamma encoding is non-linear. The most correct approach would be to linearize -> apply Rec.709 -> apply saturation -> transform to sRGB. However, this requires expensive power operations and the Rec.601 approximation provides acceptable perceptual results for real-time editing while maintaining our performance advantage of working natively in sRGB. 

*Note:* I want to add a feature later that allows use to compare the edit applied in linear- and gamma- encoded light.


## 5. Vibrance Adjustment

**What I wanted:** An adaptive saturation that boosts dull colors more than already-saturated colors, preventing oversaturation of skin tones and vivid regions.

### Algorithm 5.1: Vibrance Transform

```
Input: pixel r = [R, G, B]ᵀ, vibrance factor v ∈ ℝ
Output: transformed pixel r'

1. maxC ← max(R, G, B)
2. minC ← min(R, G, B)
3. ŝ ← (maxC - minC) / maxC        // Saturation estimate
4. f ← 1 + v(1 - ŝ)                 // Adaptive factor
5. L ← 0.299·R + 0.587·G + 0.114·B
6. r' ← [L, L, L]ᵀ + f(r - [L, L, L]ᵀ)
7. return clamp(r', 0, 255)
```

**Key insight:** Unlike saturation, the factor $f$ depends on each pixel's current saturation $\hat{s}(\mathbf{r})$, making this inherently per-pixel.

**Implementation:** See `applyVibranceGamma()` at lines 194-210 in `ImageCanvas.tsx`:
```typescript
const applyVibranceGamma = (rgb: RGB, vibrance: number): RGB => {
  if (vibrance === 0) return rgb;
  const R = rgb.r, G = rgb.g, B = rgb.b;
  const maxC = Math.max(R, G, B), minC = Math.min(R, G, B);
  const sEst = maxC === 0 ? 0 : (maxC - minC) / maxC;
  const f = 1 + vibrance * (1 - sEst);
  const gray = 0.299 * R + 0.587 * G + 0.114 * B;
  if (R === G && G === B) return { r: R, g: G, b: B };
  return { 
    r: clamp(gray + (R - gray) * f), 
    g: clamp(gray + (G - gray) * f), 
    b: clamp(gray + (B - gray) * f) 
  };
};
```

### Proof of Non-Composability

Vibrance cannot be expressed as a global affine transform (matrix + offset).

**Proof by contradiction:**

Assume vibrance can be expressed as $V(\mathbf{r}) = M\mathbf{r} + \mathbf{o}$ for some fixed $M$ and $\mathbf{o}$.

Consider two pixels:
- $\mathbf{r}_1 = [255, 0, 0]^T$ (saturated red): $\hat{s}_1 = 1$, so $f_1 = 1 + v(1-1) = 1$
- $\mathbf{r}_2 = [200, 195, 190]^T$ (desaturated): $\hat{s}_2 \approx 0.05$, so $f_2 = 1 + v(0.95) \approx 1 + 0.95v$

For $v = 1$, we have $f_1 = 1$ and $f_2 \approx 1.95$.

If vibrance were a linear transform, the ratio of how much each pixel changes would be constant:
$$\frac{\|\mathbf{r}'_1 - L_1\mathbf{1}\|}{\|\mathbf{r}_1 - L_1\mathbf{1}\|} = \frac{\|\mathbf{r}'_2 - L_2\mathbf{1}\|}{\|\mathbf{r}_2 - L_2\mathbf{1}\|}$$

But by construction:
$$\frac{\|\mathbf{r}'_1 - L_1\mathbf{1}\|}{\|\mathbf{r}_1 - L_1\mathbf{1}\|} = f_1 = 1 \neq 1.95 \approx f_2 = \frac{\|\mathbf{r}'_2 - L_2\mathbf{1}\|}{\|\mathbf{r}_2 - L_2\mathbf{1}\|}$$

This contradicts linearity. Therefore, vibrance cannot be a global affine transform. 




## 6. Hue Rotation

**What I wanted:** A transform that rotates colors around the gray axis in RGB space, changing hue while preserving brightness and leaving grays unchanged.

### Algorithm 6.1: Hue Rotation

```
Input: pixel r = [R, G, B]ᵀ, angle θ (in degrees)
Output: rotated pixel r'

1. Convert θ to radians: θ_rad ← θ · π/180
2. Compute rotation matrix R(θ) using Rodrigues' formula
3. r' ← R(θ) · r
4. return clamp(r', 0, 255)
```

**Mathematical form:** I use Rodrigues' rotation formula for rotation by angle $\theta$ around the normalized gray axis $\mathbf{u} = \frac{1}{\sqrt{3}}[1, 1, 1]^T$:

$$R(\theta) = \cos\theta \cdot I + (1-\cos\theta)(\mathbf{u}\mathbf{u}^T) + \sin\theta \cdot [\mathbf{u}]_\times$$

where $[\mathbf{u}]_\times$ is the skew-symmetric cross-product matrix:

$$[\mathbf{u}]_\times = \frac{1}{\sqrt{3}}\begin{bmatrix}0 & -1 & 1\\1 & 0 & -1\\-1 & 1 & 0\end{bmatrix}$$

After algebraic simplification, this becomes:

$$R(\theta) = \begin{bmatrix}
c + \frac{1-c}{3} & \frac{1-c}{3} - \frac{s}{\sqrt{3}} & \frac{1-c}{3} + \frac{s}{\sqrt{3}} \\
\frac{1-c}{3} + \frac{s}{\sqrt{3}} & c + \frac{1-c}{3} & \frac{1-c}{3} - \frac{s}{\sqrt{3}} \\
\frac{1-c}{3} - \frac{s}{\sqrt{3}} & \frac{1-c}{3} + \frac{s}{\sqrt{3}} & c + \frac{1-c}{3}
\end{bmatrix}$$

where $c = \cos\theta$ and $s = \sin\theta$.

**Implementation:** See `buildHueMatrix()` at lines 98-123 in `ImageCanvas.tsx`:
```typescript
const buildHueMatrix = (value: number): number[] => {
  if (value === 0) return [1, 0, 0, 0, 1, 0, 0, 0, 1];
  
  const angle = (value * Math.PI) / 180;
  const cosA = Math.cos(angle);
  const sinA = Math.sin(angle);
  
  return [
    cosA + (1 - cosA) / 3,
    1/3 * (1 - cosA) - Math.sqrt(1/3) * sinA,
    1/3 * (1 - cosA) + Math.sqrt(1/3) * sinA,
    1/3 * (1 - cosA) + Math.sqrt(1/3) * sinA,
    cosA + 1/3 * (1 - cosA),
    1/3 * (1 - cosA) - Math.sqrt(1/3) * sinA,
    1/3 * (1 - cosA) - Math.sqrt(1/3) * sinA,
    1/3 * (1 - cosA) + Math.sqrt(1/3) * sinA,
    cosA + 1/3 * (1 - cosA)
  ];
};
```

### Proof of Properties

**Claim 1:** Any gray pixel $\mathbf{g} = [g, g, g]^T$ remains unchanged under hue rotation.

**Proof:**

By definition of rotation, any vector parallel to the rotation axis is unchanged. Since $\mathbf{g} = g[1,1,1]^T = g\sqrt{3} \cdot \mathbf{u}$ is parallel to $\mathbf{u}$:

$$R(\theta)\mathbf{g} = R(\theta)(g\sqrt{3}\mathbf{u}) = g\sqrt{3} \cdot R(\theta)\mathbf{u} = g\sqrt{3} \cdot \mathbf{u} = \mathbf{g}$$

The last equality follows because rotating a vector around itself leaves it unchanged. ∎

**Claim 2:** $R(\theta)$ is an orthogonal matrix, preserving distances and angles.

**Proof:**

Rodrigues' formula constructs rotation matrices by definition. All rotation matrices satisfy the orthogonality property $R(\theta)^T R(\theta) = I$.

This can be verified directly from the formula: the three components $\cos\theta \cdot I$, $(1-\cos\theta)(\mathbf{u}\mathbf{u}^T)$, and $\sin\theta \cdot [\mathbf{u}]_\times$ are constructed such that their combination produces an orthogonal matrix.

As a consequence, for any two pixels $\mathbf{r}_1, \mathbf{r}_2$:
$$\|\mathbf{r}'_1 - \mathbf{r}'_2\| = \|R(\mathbf{r}_1 - \mathbf{r}_2)\| = \|\mathbf{r}_1 - \mathbf{r}_2\|$$

Hue rotation preserves all distances in RGB space. ∎

*Note:* Hue rotation is a geometric operation (it works identically in linear or gamma) encoded RGB space since it's just a 3D rotation. The operation is space-independent.



## 7. Whites and Blacks Adjustments

**What I wanted:** Two tone curve adjustments that target specific tonal ranges—whites for bright tones and blacks for dark tones—with smooth parametric transitions. These adjustments allow precise control over highlights and shadows while preserving detail in other tonal ranges, similar to Lightroom's Whites and Blacks sliders.

### Algorithm 7.1: Smoothstep Function

The smoothstep function provides a smooth S-curve transition between two edge values:

```
Input: edge values edge₀, edge₁ ∈ ℝ, input x ∈ ℝ
Output: smoothstep weight w ∈ [0, 1]

1. t ← clamp((x - edge₀)/(edge₁ - edge₀), 0, 1)
2. w ← t² × (3 - 2t)
3. return w
```

**Mathematical form:** 

$$\text{smoothstep}(x) = t^2(3 - 2t), \quad \text{where } t = \text{clamp}\left(\frac{x - \text{edge}_0}{\text{edge}_1 - \text{edge}_0}, 0, 1\right)$$

The smoothstep function provides:
- Smooth, continuous transitions (C¹ continuous)
- Zero slope at the boundaries (no discontinuities)
- Perceptually pleasing S-curve shape
- Efficient computation (polynomial, no transcendental functions)

### Algorithm 7.2: Whites Transform

**What it does:** Adjusts bright tones (highlights) with smooth falloff toward midtones, preserving shadow detail.

```
Input: pixel r = [R, G, B]ᵀ, whites adjustment value w ∈ ℝ
Output: transformed pixel r'

1. L ← (0.299·R + 0.587·G + 0.114·B) / 255    // Normalized luminance
2. weight ← smoothstep(0.4, 0.8, L)            // Weight: high for bright, low for dark
3. adjustment ← w × weight
4. R' ← clamp(R + adjustment, 0, 255)
5. G' ← clamp(G + adjustment, 0, 255)
6. B' ← clamp(B + adjustment, 0, 255)
7. return [R', G', B']ᵀ
```

**Key properties:**
- Pixels with L < 0.4 get minimal adjustment (weight ≈ 0)
- Pixels with L > 0.8 get full adjustment (weight ≈ 1)
- Pixels in the 0.4-0.8 range get smooth transition
- The adjustment is applied uniformly to all channels, preserving color relationships

**Implementation:** See `applyWhites()` at lines 266-279 in `ImageCanvas.tsx`:

```typescript
const applyWhites = (rgb: RGB, value: number): RGB => {
  if (value === 0) return rgb;
  const luminance = (0.299 * rgb.r + 0.587 * rgb.g + 0.114 * rgb.b) / 255;
  const weight = smoothstep(0.4, 0.8, luminance);
  const adjustment = value * weight;
  return {
    r: clamp(rgb.r + adjustment),
    g: clamp(rgb.g + adjustment),
    b: clamp(rgb.b + adjustment)
  };
};
```

### Algorithm 7.3: Blacks Transform

**What it does:** Adjusts dark tones (shadows) with smooth falloff toward midtones, preserving highlight detail.

```
Input: pixel r = [R, G, B]ᵀ, blacks adjustment value b ∈ ℝ
Output: transformed pixel r'

1. L ← (0.299·R + 0.587·G + 0.114·B) / 255    // Normalized luminance
2. weight ← smoothstep(0.8, 0.2, L)            // Inverted: high for dark, low for bright
3. adjustment ← b × weight
4. R' ← clamp(R + adjustment, 0, 255)
5. G' ← clamp(G + adjustment, 0, 255)
6. B' ← clamp(B + adjustment, 0, 255)
7. return [R', G', B']ᵀ
```

**Key properties:**
- Pixels with L < 0.2 get full adjustment (weight ≈ 1)
- Pixels with L > 0.8 get minimal adjustment (weight ≈ 0)
- Pixels in the 0.2-0.8 range get smooth transition
- The inverted smoothstep (edge₀ > edge₁) creates a curve that's high for dark pixels

**Implementation:** See `applyBlacks()` at lines 281-294 in `ImageCanvas.tsx`:

```typescript
const applyBlacks = (rgb: RGB, value: number): RGB => {
  if (value === 0) return rgb;
  const luminance = (0.299 * rgb.r + 0.587 * rgb.g + 0.114 * rgb.b) / 255;
  const weight = smoothstep(0.8, 0.2, luminance);
  const adjustment = value * weight;
  return {
    r: clamp(rgb.r + adjustment),
    g: clamp(rgb.g + adjustment),
    b: clamp(rgb.b + adjustment)
  };
};
```

### Proof of Non-Composability

**Claim:** Whites and blacks adjustments cannot be expressed as global affine transforms (matrix + offset).

**Proof by contradiction:**

Assume whites adjustment can be expressed as $W(\mathbf{r}) = M\mathbf{r} + \mathbf{o}$ for some fixed $M$ and $\mathbf{o}$.

Consider two pixels with different luminances:
- $\mathbf{r}_1$ with $L_1 = 0.9$ (bright): weight $w_1 \approx 1$, adjustment $\approx v$
- $\mathbf{r}_2$ with $L_2 = 0.3$ (dark): weight $w_2 \approx 0$, adjustment $\approx 0$

If whites were a linear transform, the adjustment would be constant for all pixels. But by construction, the adjustment depends on each pixel's luminance, which varies per pixel. Therefore, whites (and similarly blacks) cannot be global affine transforms and must be applied per-pixel.

### Design Decisions

1. **Smoothstep over linear interpolation:** Smoothstep provides perceptually smooth transitions with zero slope at boundaries, avoiding visible discontinuities that linear interpolation would create.

2. **Luminance-based weighting:** Using Rec.601 luminance weights (0.299, 0.587, 0.114) ensures the adjustment respects perceived brightness, affecting pixels based on how bright they appear to human vision.

3. **Uniform channel adjustment:** The adjustment is applied equally to R, G, B channels, preserving color relationships (hue and saturation) while only affecting brightness in the targeted tonal range.

4. **Edge value selection:** 
   - Whites: edge₀=0.4, edge₁=0.8 targets the 40-80% luminance range, affecting highlights while preserving midtones and shadows
   - Blacks: edge₀=0.8, edge₁=0.2 (inverted) targets the 20-80% luminance range, affecting shadows while preserving midtones and highlights

5. **Per-pixel processing:** Like vibrance, whites and blacks are inherently per-pixel operations because the weight depends on each pixel's luminance value, making them non-composable with global affine transforms.



## 8. Transform Composition

I realized that brightness, contrast, saturation, and hue are all affine transforms (matrix + offset). By composing them algebraically before processing pixels, I can apply multiple transforms with a single matrix multiplication per pixel.

### Algorithm 8.1: Affine Transform Composition

```
Input: List of transforms T₁, T₂, ..., Tₙ where Tᵢ = (Mᵢ, oᵢ)
Output: Composed transform T_composed = (M_composed, o_composed)

1. M ← M₁
2. o ← o₁
3. for i = 2 to n:
4.   M ← Mᵢ · M          // Matrix multiplication
5.   o ← Mᵢ · o + oᵢ      // Transform offset through matrix
6. return (M, o)
```

**Mathematical justification:** Composing two affine transforms:
$$\mathbf{r}'' = M_2(M_1\mathbf{r} + \mathbf{o}_1) + \mathbf{o}_2 = (M_2 M_1)\mathbf{r} + (M_2\mathbf{o}_1 + \mathbf{o}_2)$$

So the composed transform is $(M_2 M_1, M_2\mathbf{o}_1 + \mathbf{o}_2)$.

**Implementation:** See `composeAffineTransforms()` at lines 235-286 in `ImageCanvas.tsx`:
```typescript
const composeAffineTransforms = (
  transforms: Array<{ matrix: number[]; offset: number[] }>
): { matrix: number[]; offset: number[] } => {
  if (transforms.length === 0) {
    return { matrix: [1, 0, 0, 0, 1, 0, 0, 0, 1], offset: [0, 0, 0] };
  }
  
  if (transforms.length === 1) {
    return transforms[0];
  }
  
  let resultMatrix = [...transforms[0].matrix];
  let resultOffset = [...transforms[0].offset];
  
  for (let i = 1; i < transforms.length; i++) {
    const M2 = transforms[i].matrix;
    const o2 = transforms[i].offset;
    
    // Matrix multiplication: M_composed = M2 * M1
    const newMatrix = [
      M2[0] * resultMatrix[0] + M2[1] * resultMatrix[3] + M2[2] * resultMatrix[6],
      M2[0] * resultMatrix[1] + M2[1] * resultMatrix[4] + M2[2] * resultMatrix[7],
      M2[0] * resultMatrix[2] + M2[1] * resultMatrix[5] + M2[2] * resultMatrix[8],
      M2[3] * resultMatrix[0] + M2[4] * resultMatrix[3] + M2[5] * resultMatrix[6],
      M2[3] * resultMatrix[1] + M2[4] * resultMatrix[4] + M2[5] * resultMatrix[7],
      M2[3] * resultMatrix[2] + M2[4] * resultMatrix[5] + M2[5] * resultMatrix[8],
      M2[6] * resultMatrix[0] + M2[7] * resultMatrix[3] + M2[8] * resultMatrix[6],
      M2[6] * resultMatrix[1] + M2[7] * resultMatrix[4] + M2[8] * resultMatrix[7],
      M2[6] * resultMatrix[2] + M2[7] * resultMatrix[5] + M2[8] * resultMatrix[8]
    ];
    
    // Offset transformation: o_composed = M2 * o1 + o2
    const newOffset = [
      M2[0] * resultOffset[0] + M2[1] * resultOffset[1] + M2[2] * resultOffset[2] + o2[0],
      M2[3] * resultOffset[0] + M2[4] * resultOffset[1] + M2[5] * resultOffset[2] + o2[1],
      M2[6] * resultOffset[0] + M2[7] * resultOffset[1] + M2[8] * resultOffset[2] + o2[2]
    ];
    
    resultMatrix = newMatrix;
    resultOffset = newOffset;
  }
  
  return { matrix: resultMatrix, offset: resultOffset };
};
```


### Proof of Correctness

For transforms $T_1, T_2, \ldots, T_n$, Algorithm 8.1 produces a transform $T$ such that:
$$T(\mathbf{r}) = T_n(T_{n-1}(\cdots T_2(T_1(\mathbf{r})) \cdots))$$

**Proof by induction:**

*Base case ($n=1$):* Trivial—the single transform is returned as-is.

*Inductive step:* Assume correctness for $n-1$ transforms. Let $(M', \mathbf{o}')$ be the composition of $T_1, \ldots, T_{n-1}$. After iteration $n$:
$$M = M_n M', \quad \mathbf{o} = M_n\mathbf{o}' + \mathbf{o}_n$$

Applying this to pixel $\mathbf{r}$:
$$T(\mathbf{r}) = M\mathbf{r} + \mathbf{o} = M_n M'\mathbf{r} + M_n\mathbf{o}' + \mathbf{o}_n$$
$$= M_n(M'\mathbf{r} + \mathbf{o}') + \mathbf{o}_n = T_n(T'(\mathbf{r}))$$

where $T'$ is the composition of $T_1, \ldots, T_{n-1}$. By the inductive hypothesis, this equals the full composition. ∎



## 9. Processing Pipeline

I scan through the transform order and batch all consecutive affine transforms, then apply per-pixel transforms individually when encountered. Convolution operations are applied separately as they operate on spatial neighborhoods rather than individual pixels.

### Algorithm 9.1: Smart Batching Pipeline

```
Input: Pixel array P (as ImageData), ordered transform list L
Output: Transformed pixel array P'

1. i ← 0
2. while i < length(L):
3.   batch ← []
4.   while i < length(L) and L[i] is affine:  // Collect affine transforms
5.     batch.append(L[i])
6.     i ← i + 1
7.   if batch is not empty:
8.     (M, o) ← composeAffineTransforms(batch)
9.     for each pixel p in P:
10.      p ← M · p + o
11.   if i < length(L) and L[i] is per-pixel affine:    // Apply per-pixel transform
12.     for each pixel p in P:
13.       p ← L[i].apply(p)
14.     i ← i + 1
15.   if i < length(L) and L[i] is convolution:    // Apply convolution operation
16.     P ← L[i].convolve(P)  // Operates on entire ImageData
17.     i ← i + 1
18. return P
```

**Why this works:**
- Affine transforms (brightness, contrast, saturation, hue) can be batched because they apply the same matrix to every pixel
- Per-pixel transforms (vibrance, whites, blacks) adapt per pixel, so they must be applied individually
- Convolution operations (blur, sharpen, edge, denoise, customConv) operate on spatial neighborhoods and must process the entire ImageData structure to access neighboring pixels
- By batching consecutive affine transforms, I minimize per-pixel operations

**Implementation:** See `ImageCanvas.tsx` lines 604-651. The pipeline processes transforms in order:

```typescript
for (const inst of pipeline) {
  if (!inst.enabled) continue;
  if (inst.kind === 'brightness' || inst.kind === 'contrast' || 
      inst.kind === 'saturation' || inst.kind === 'hue' || 
      inst.kind === 'vibrance') {
    // Affine transforms: batch and compose
    const batch: Array<{ matrix: number[]; offset: number[] }> = [];
    if (kind === 'brightness') batch.push(buildBrightnessMatrix(...));
    if (kind === 'contrast') batch.push(buildContrastMatrix(...));
    // ... compose and apply ...
  } else if (inst.kind === 'blur') {
    const out = cpuConvolutionBackend.blur(imageData, p);
    for (let j = 0; j < data.length; j++) data[j] = out.data[j];
  } else if (inst.kind === 'sharpen') {
    const out = cpuConvolutionBackend.sharpen(imageData, p);
    // ... apply result ...
  }
  // ... other convolution operations ...
}
```

**Key Design Decision:** Convolutions are applied to the entire `ImageData` object rather than pixel-by-pixel because:
1. They require access to neighboring pixels, which is naturally handled by the `ImageData` structure
2. The convolution backend (`cpuConvolutionBackend`) operates on complete images, enabling optimizations like stride and efficient boundary handling
3. This separation keeps the convolution logic modular and allows for future GPU acceleration

**Complexity analysis:**
Let $n$ be the number of transforms, $p$ be the number of pixels, and $k$ be the average kernel size for convolutions.

Without batching: $O(np)$ matrix-vector multiplications + $O(nkp)$ convolution operations

With batching: $O(n^2)$ matrix-matrix multiplications (negligible for small $n$) + $O(bp)$ matrix-vector multiplications (where $b$ is the number of batches) + $O(nkp)$ convolution operations

For typical image processing where $n \ll \sqrt{p}$ (e.g., 5 transforms on a 2-megapixel image), batching provides significant performance gains for affine transforms. Convolutions remain $O(kp)$ per operation regardless of batching, as they are inherently spatial operations.

**Order Dependency:**

The pipeline applies transforms in the order specified by the user. This order matters because:
- Affine transforms are linear and commute: $T_1(T_2(\mathbf{r})) = T_2(T_1(\mathbf{r}))$ for affine $T_1, T_2$
- However, convolutions do not commute with affine transforms or other convolutions in general
- For example, blurring then sharpening produces different results than sharpening then blurring

Therefore, the pipeline respects user-specified order and applies each transform sequentially to the current image state.


## 10. Convolution Operations

**What I wanted:** A framework for applying spatial filters to images through discrete convolution, enabling blur, sharpening, edge detection, and denoising operations.

Unlike the affine color transforms (brightness, contrast, saturation, hue), convolutions operate on local neighborhoods of pixels. Each output pixel depends on a weighted sum of its surrounding pixels, making these operations inherently spatial and non-composable with global affine transforms.

### 9.1 Mathematical Foundation

**Discrete 2D Convolution:**

For an image $I(x, y)$ and kernel $K(i, j)$ of size $n \times n$ (typically odd), the convolution at pixel $(x, y)$ is:

$$(I * K)(x, y) = \sum_{i=-h}^{h} \sum_{j=-h}^{h} I(x + i, y + j) \cdot K(h + i, h + j)$$

where $h = \lfloor n/2 \rfloor$ is the kernel half-size, and $K$ is centered at $(h, h)$.

**Boundary Handling:**

Since convolution requires accessing pixels outside image boundaries, we implement three padding modes:

1. **Zero padding:** $I(x, y) = 0$ for out-of-bounds coordinates
2. **Edge padding:** $I(x, y) = I(\text{clamp}(x, 0, W-1), \text{clamp}(y, 0, H-1))$
3. **Reflect padding:** $I(x, y) = I(\text{reflect}(x, W), \text{reflect}(y, H))$ where reflection mirrors coordinates across boundaries

**Implementation:** See `padIndex()` at lines 15-26 in `convolution.ts`:
```typescript
function padIndex(i: number, limit: number, mode: PaddingMode): number {
  if (i >= 0 && i < limit) return i;
  if (mode === 'zero') return -1; // sentinel for zero
  if (mode === 'edge') return i < 0 ? 0 : limit - 1;
  // reflect
  let idx = i;
  if (idx < 0) idx = -idx - 1;
  const period = (limit - 1) * 2;
  idx = idx % period;
  if (idx >= limit) idx = period - idx;
  return idx;
}
```

### 9.2 Per-Pixel Convolution

**Algorithm 10.1: Convolve at Single Pixel**

```
Input: source ImageData, pixel coordinates (x, y), kernel K[n×n], params
Output: transformed RGB [R', G', B']ᵀ

1. h ← ⌊n/2⌋
2. rAcc ← 0, gAcc ← 0, bAcc ← 0
3. for ky = 0 to n-1:
4.   for kx = 0 to n-1:
5.     ix ← x + (kx - h) · dilation
6.     iy ← y + (ky - h) · dilation
7.     (sx, sy) ← padIndex(ix, width, padding), padIndex(iy, height, padding)
8.     if (sx, sy) is out-of-bounds (zero padding): continue
9.     w ← K[ky][kx]
10.    (R, G, B) ← source[sy · width + sx]
11.    if perChannel:
12.      rAcc ← rAcc + R · w
13.      gAcc ← gAcc + G · w
14.      bAcc ← bAcc + B · w
15.    else:  // luminance-only
16.      L ← 0.299·R + 0.587·G + 0.114·B
17.      rAcc ← rAcc + L · w
18.      gAcc ← gAcc + L · w
19.      bAcc ← bAcc + L · w
20. return [clamp(rAcc, 0, 255), clamp(gAcc, 0, 255), clamp(bAcc, 0, 255)]
```

**Implementation:** See `convolveAtPixel()` at lines 28-72 in `convolution.ts`:
```typescript
export function convolveAtPixel(
  source: ImageData,
  x: number,
  y: number,
  kernel: number[][],
  params: ConvolutionCommonParams & { perChannel?: boolean }
): [number, number, number] {
  const { width, height, data } = source;
  const k = kernel;
  const kSize = k.length;
  const kHalf = Math.floor(kSize / 2);
  const dilation = params.dilation ?? 1;
  const padding: PaddingMode = params.padding ?? 'edge';
  const perChannel = params.perChannel ?? true;

  let rAcc = 0, gAcc = 0, bAcc = 0;
  for (let ky = 0; ky < kSize; ky++) {
    for (let kx = 0; kx < kSize; kx++) {
      const ix = x + (kx - kHalf) * dilation;
      const iy = y + (ky - kHalf) * dilation;
      let sx = padIndex(ix, width, padding);
      let sy = padIndex(iy, height, padding);
      const w = k[ky][kx];
      if (sx === -1 || sy === -1) {
        continue; // zero padding
      }
      const idx = (sy * width + sx) * 4;
      const R = data[idx], G = data[idx + 1], B = data[idx + 2];
      if (perChannel) {
        rAcc += R * w;
        gAcc += G * w;
        bAcc += B * w;
      } else {
        const gray = 0.299 * R + 0.587 * G + 0.114 * B;
        rAcc += gray * w;
        gAcc += gray * w;
        bAcc += gray * w;
      }
    }
  }
  return [clamp255(rAcc), clamp255(gAcc), clamp255(bAcc)];
}
```

**Design Decision:** By default, we apply convolution per-channel (perChannel = true), treating R, G, B independently. This preserves color relationships better than luminance-only processing, which would desaturate edges and features. The luminance-only mode is available for specialized use cases.

### 9.3 Blur Operations

**Gaussian Blur:**

Gaussian blur uses a kernel derived from the 2D Gaussian function:

$$G(x, y) = \frac{1}{2\pi\sigma^2} \exp\left(-\frac{x^2 + y^2}{2\sigma^2}\right)$$

For a discrete $n \times n$ kernel centered at $(0, 0)$:

$$K[i][j] = \exp\left(-\frac{(i-h)^2 + (j-h)^2}{2\sigma^2}\right)$$

where $h = \lfloor n/2 \rfloor$. The kernel is normalized so $\sum_{i,j} K[i][j] = 1$ to preserve image brightness.

**Box Blur:**

Box blur uses a uniform kernel:

$$K[i][j] = \frac{1}{n^2} \quad \forall i, j \in [0, n-1]$$

Box blur is computationally cheaper than Gaussian blur but produces less smooth results with visible artifacts.

**Implementation:** See `gaussianKernel()` and `boxKernel()` at lines 112-136 in `convolution.ts`:
```typescript
export function gaussianKernel(size: 3 | 5 | 7, sigma?: number): number[][] {
  const s = sigma ?? (size === 3 ? 0.85 : size === 5 ? 1.2 : 1.6);
  const half = Math.floor(size / 2);
  const k: number[][] = [];
  let sum = 0;
  for (let y = -half; y <= half; y++) {
    const row: number[] = [];
    for (let x = -half; x <= half; x++) {
      const v = Math.exp(-(x * x + y * y) / (2 * s * s));
      row.push(v);
      sum += v;
    }
    k.push(row);
  }
  // normalize
  for (let y = 0; y < size; y++) {
    for (let x = 0; x < size; x++) k[y][x] /= sum;
  }
  return k;
}
```

### 9.4 Sharpening Operations

**Unsharp Masking:**

Unsharp masking enhances edges by subtracting a blurred version from the original:

$$\text{sharpened} = \text{original} + \alpha(\text{original} - \text{blurred}) = (1+\alpha)\text{original} - \alpha \cdot \text{blurred}$$

This can be expressed as a single convolution kernel:

$$K_{\text{unsharp}} = (1+\alpha)\delta - \alpha \cdot K_{\text{blur}}$$

where $\delta$ is the identity (delta) kernel (1 at center, 0 elsewhere).

**Laplacian Sharpening:**

The Laplacian operator detects edges by computing the second derivative. For sharpening:

$$\text{sharpened} = \text{original} + \alpha \cdot \text{Laplacian}(\text{original})$$

The discrete Laplacian kernel (3×3) is:

$$K_{\text{Laplacian}} = \begin{bmatrix}
0 & -\alpha & 0 \\
-\alpha & 1+4\alpha & -\alpha \\
0 & -\alpha & 0
\end{bmatrix}$$

**Implementation:** See `unsharpKernel()` and `laplacianKernel()` at lines 166-184 in `convolution.ts`:
```typescript
export function unsharpKernel(amount: number, size: 3 | 5): number[][] {
  const blur = size === 3 ? boxKernel(3) : boxKernel(5);
  const k = blur.map(row => row.map(v => -amount * v));
  const c = Math.floor(size / 2);
  k[c][c] += 1 + amount;
  return k;
}
```

### 9.5 Edge Detection

**Gradient-Based Edge Detection:**

Edge detection computes the image gradient $\nabla I = [\frac{\partial I}{\partial x}, \frac{\partial I}{\partial y}]^T$. The gradient magnitude indicates edge strength:

$$|\nabla I| = \sqrt{\left(\frac{\partial I}{\partial x}\right)^2 + \left(\frac{\partial I}{\partial y}\right)^2}$$

**Sobel Operator:**

The Sobel operator approximates derivatives using:

$$G_x = \begin{bmatrix}
-1 & 0 & 1 \\
-2 & 0 & 2 \\
-1 & 0 & 1
\end{bmatrix}, \quad
G_y = \begin{bmatrix}
-1 & -2 & -1 \\
0 & 0 & 0 \\
1 & 2 & 1
\end{bmatrix}$$

The gradient components are computed via convolution:
- $G_x = I * K_x$ (horizontal gradient)
- $G_y = I * K_y$ (vertical gradient)

The magnitude is: $|\nabla I| = \sqrt{G_x^2 + G_y^2}$

**Prewitt Operator:**

Similar to Sobel but with uniform weights:

$$G_x = \begin{bmatrix}
-1 & 0 & 1 \\
-1 & 0 & 1 \\
-1 & 0 & 1
\end{bmatrix}, \quad
G_y = \begin{bmatrix}
-1 & -1 & -1 \\
0 & 0 & 0 \\
1 & 1 & 1
\end{bmatrix}$$

**Implementation:** See `applyEdge()` at lines 211-237 in `convolution.ts`:
```typescript
export function applyEdge(imageData: ImageData, params: EdgeParams): ImageData {
  const { width, height, data } = imageData;
  const out = new ImageData(width, height);
  const { kx, ky } = params.operator === 'sobel' ? sobelKernels() : prewittKernels();
  const perChannel = true;
  const padding: PaddingMode = params.padding ?? 'edge';
  for (let y = 0; y < height; y++) {
    for (let x = 0; x < width; x++) {
      const [rx, gx, bx] = convolveAtPixel(imageData, x, y, kx, { padding, perChannel });
      const [ry, gy, by] = convolveAtPixel(imageData, x, y, ky, { padding, perChannel });
      let r = 0, g = 0, b = 0;
      if (params.combine === 'x') {
        r = Math.abs(rx); g = Math.abs(gx); b = Math.abs(bx);
      } else if (params.combine === 'y') {
        r = Math.abs(ry); g = Math.abs(gy); b = Math.abs(by);
      } else {
        r = Math.hypot(rx, ry); g = Math.hypot(gx, gy); b = Math.hypot(bx, by);
      }
      const idx = (y * width + x) * 4;
      out.data[idx] = clamp255(r);
      out.data[idx + 1] = clamp255(g);
      out.data[idx + 2] = clamp255(b);
      out.data[idx + 3] = data[idx + 3];
    }
  }
  return out;
}
```

**Key insight:** Edge detection applies two separate convolutions (horizontal and vertical gradients), then combines them. This is fundamentally different from a single-kernel convolution and cannot be reduced to a single matrix operation.

### 9.6 Denoising Operations

**Mean Filter (Box Blur):**

Mean filtering replaces each pixel with the average of its neighborhood, reducing noise but also blurring edges:

$$\text{denoised}(x, y) = \frac{1}{n^2} \sum_{i=-h}^{h} \sum_{j=-h}^{h} I(x+i, y+j)$$

We optionally blend the filtered result with the original:

$$\text{output} = (1-k) \cdot \text{original} + k \cdot \text{filtered}$$

where $k \in [0, 1]$ is the strength parameter.

**Median Filter:**

Median filtering replaces each pixel with the median of its neighborhood. Unlike mean filtering, median filtering preserves edges better while removing salt-and-pepper noise:

$$\text{denoised}(x, y) = \text{median}\{I(x+i, y+j) : i, j \in [-h, h]\}$$

**Implementation:** See `applyDenoise()` at lines 239-293 in `convolution.ts`. The median filter implementation collects all pixels in the neighborhood, sorts them, and selects the middle value:
```typescript
// median filter
const windowR = new Array<number>(kSize * kSize);
const windowG = new Array<number>(kSize * kSize);
const windowB = new Array<number>(kSize * kSize);
// ... collect pixels ...
windowR.sort((a, b) => a - b);
windowG.sort((a, b) => a - b);
windowB.sort((a, b) => a - b);
const mid = Math.floor(windowR.length / 2);
out.data[idxOut] = windowR[mid];
```

**Proof of Edge Preservation (Median Filter):**

**Claim:** Median filtering preserves step edges better than mean filtering.

**Proof:** Consider a step edge where pixels transition from value $a$ to value $b$ with $a < b$. 

For mean filtering: If the kernel straddles the edge, the output is a weighted average of $a$ and $b$, producing an intermediate value that blurs the edge.

For median filtering: If more than half the kernel pixels are on one side of the edge, the median will be either $a$ or $b$ (depending on which side dominates), preserving the sharp transition. Only when the kernel is exactly centered on the edge does the median produce an intermediate value.

Therefore, median filtering preserves edges more effectively than mean filtering. ∎

### 9.7 Custom Convolution

Custom convolution allows users to define arbitrary kernels, enabling experimentation with novel filters. The implementation applies the user-provided kernel directly via `convolveImageData()`.

**Implementation:** See `applyCustomConv()` at lines 295-301 in `convolution.ts`:
```typescript
export function applyCustomConv(imageData: ImageData, params: CustomConvParams): ImageData {
  return convolveImageData(imageData, params.kernel, { 
    stride: params.stride ?? 1, 
    padding: params.padding ?? 'edge', 
    perChannel: true 
  });
}
```

### 9.8 Stride and Dilation

**Stride:**

Stride controls the spacing between output pixels. With stride $s$, we compute convolution only at positions $(x, y)$ where $x \equiv 0 \pmod{s}$ and $y \equiv 0 \pmod{s}$. For stride $> 1$, we fill skipped pixels using nearest-neighbor interpolation to avoid black gaps.

**Dilation:**

Dilation spaces out kernel elements. With dilation $d$, kernel element $K[i][j]$ is applied at offset $(i \cdot d, j \cdot d)$ instead of $(i, j)$. This effectively enlarges the kernel's receptive field without increasing its size.

**Implementation:** See `convolveImageData()` at lines 74-110 in `convolution.ts` for stride handling, and `convolveAtPixel()` line 47 for dilation.

### 9.9 Non-Composability with Affine Transforms

**Claim:** Convolution operations cannot be expressed as global affine transforms (matrix + offset).

**Proof by contradiction:**

Assume a convolution operation $C$ can be expressed as $C(\mathbf{r}) = M\mathbf{r} + \mathbf{o}$ for some fixed $3 \times 3$ matrix $M$ and offset $\mathbf{o}$.

Consider two pixels $\mathbf{r}_1$ and $\mathbf{r}_2$ that are identical in value but have different spatial contexts (different neighbors). Under a convolution, these pixels will produce different outputs because convolution depends on local neighborhoods. However, under an affine transform:

$$C(\mathbf{r}_1) = M\mathbf{r}_1 + \mathbf{o} = M\mathbf{r}_2 + \mathbf{o} = C(\mathbf{r}_2)$$

This contradicts the spatial dependency of convolution. Therefore, convolutions cannot be global affine transforms and must be applied separately in the processing pipeline. 

# References

Bezryadin, S., Bourov, P., & Ilinih, D. (2007, January). Brightness calculation in digital image processing. In International symposium on technologies for digital photo fulfillment (Vol. 1, pp. 10-15). Society for Imaging Science and Technology. https://library.imaging.org/admin/apis/public/api/ist/website/downloadArticle/tdpf/1/1/art00005

Convolution. Wikipedia. https://en.wikipedia.org/wiki/Convolution

Czech Technical University in Prague. (n.d.). Image Processing Fundamentals - Convolution-based Operations. https://cmp.felk.cvut.cz/cmp/courses/dzo/resources/course_ip_tudelft/course/fip-Convolut-2.html

Discrete Laplace operator. Wikipedia. https://en.wikipedia.org/wiki/Discrete_Laplace_operator

Huamán, A. (n.d.). Changing the contrast and brightness of an image!. OpenCV. https://docs.opencv.org/4.x/d3/dc1/tutorial_basic_linear_transform.html

Intel Corporation. (2022). Two-Dimensional Finite Linear Convolution. Intel® Integrated Performance Primitives Developer Guide. https://www.intel.com/content/www/us/en/docs/ipp/developer-guide-reference/2022-1/convolution.html

Kernel (image processing). Wikipedia. https://en.wikipedia.org/wiki/Kernel_%28image_processing%29

Lisitsa, N. (2024, October 10). Transforming colors with matrices. lisyarus blog. https://lisyarus.github.io/blog/posts/transforming-colors-with-matrices.html

McNair, M. (n.d.). Color Correction. https://maximmcnair.com/p/webgl-color-correction

Meyer, M. (2009, January 19). Analyzing photoshop vibrance and saturation. Alaska Photographer Mark Meyer. https://www.photo-mark.com/notes/analyzing-photoshop-vibrance-and-saturation/

Multidimensional discrete convolution. Wikipedia. https://en.wikipedia.org/wiki/Multidimensional_discrete_convolution

NVIDIA Corporation. (n.d.). Convolution. NVIDIA Developer Documentation. https://developer.nvidia.com/discover/convolution

Prewitt operator. Wikipedia. https://en.wikipedia.org/wiki/Prewitt_operator

Relative luminance. Wikipedia. https://en.wikipedia.org/wiki/Relative_luminance

Rodrigues’ rotation formula. Wikipedia. https://en.wikipedia.org/wiki/Rodrigues%27_rotation_formula

Separable filter. Wikipedia. https://en.wikipedia.org/wiki/Separable_filter

Sobel operator. Wikipedia. https://en.wikipedia.org/wiki/Sobel_operator

sRGB. Wikipedia. https://en.wikipedia.org/wiki/SRGB

University of Edinburgh. (n.d.). Convolution. HIPR2 - Hypermedia Image Processing Reference. https://homepages.inf.ed.ac.uk/rbf/HIPR2/convolve.htm